In [ ]:
import pandas as pd

try:
    df = pd.read_csv('final_food_delivery_dataset.csv')
except:
    # Fallback to regeneration if file is missing (robustness)
    orders_df = pd.read_csv('orders.csv')
    users_df = pd.read_json('users.json')
    import re
    restaurant_data = []
    with open('restaurants.sql', 'r') as f:
        for line in f:
            if line.strip().startswith("INSERT INTO restaurants VALUES"):
                match = re.search(r"\((.*)\);", line)
                if match:
                    restaurant_data.append([p.strip().strip("'") for p in match.group(1).split(',')])
    restaurants_df = pd.DataFrame(restaurant_data, columns=['restaurant_id', 'restaurant_name', 'cuisine', 'rating'])
    restaurants_df['restaurant_id'] = pd.to_numeric(restaurants_df['restaurant_id'])
    
    merged = pd.merge(orders_df, users_df, on='user_id', how='left')
    df = pd.merge(merged, restaurants_df, on='restaurant_id', how='left', suffixes=('_order', '_details'))
    df['rating'] = pd.to_numeric(df['rating'])


# Check min and max to decide on bins
print("Min rating:", df['rating'].min())
print("Max rating:", df['rating'].max())

# Create bins. Common ranges: 3.0-3.5, 3.5-4.0, 4.0-4.5, 4.5-5.0
# Or maybe just integer buckets: 3-4, 4-5.
# Let's try 0.5 increments for better granularity.
bins = [0, 3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['Below 3.0', '3.0 - 3.5', '3.5 - 4.0', '4.0 - 4.5', '4.5 - 5.0']

# Note: pandas cut includes rightmost edge by default, so (3.0, 3.5]. 
# To make it typically intuitive (3.0 to 3.49, 3.5 to 3.99), we might want right=False, but max is 5.0 usually.
# Let's stick to standard cut and see.
# Alternatively, I can use floor.

df['rating_range'] = pd.cut(df['rating'], bins=bins, labels=labels, right=True, include_lowest=True)

rating_revenue = df.groupby('rating_range')['total_amount'].sum().sort_values(ascending=False)

print("\nTotal Revenue by Rating Range:")
print(rating_revenue)